In [1]:
import tensorflow as tf
import utils
import numpy as np

In [2]:
# load training data and labels
data0 = utils.load_h5("ply_data_train0.h5")
data1 = utils.load_h5("ply_data_train1.h5")
data2 = utils.load_h5("ply_data_train2.h5")
data3 = utils.load_h5("ply_data_train3.h5")
data4 = utils.load_h5("ply_data_train4.h5")

# train_data = data0[0]
# print(np.shape(train_data))

# train_labels = data0[1]
# catagory_names = utils.get_category_names()
# print(np.shape(train_labels))

In [3]:
# aggregate training data, training label
train_data = np.append(data0[0], data1[0], axis=0)
train_data = np.append(train_data, data2[0], axis=0)
train_data = np.append(train_data, data3[0], axis=0)
train_data = np.append(train_data, data4[0], axis=0)
print(np.shape(train_data))

train_labels = np.append(data0[1], data1[1], axis=0)
train_labels = np.append(train_labels, data2[1], axis=0)
train_labels = np.append(train_labels, data3[1], axis=0)
train_labels = np.append(train_labels, data4[1], axis=0)
print(np.shape(train_labels))

(9840, 2048, 3)
(9840, 1)


In [4]:
# load test data
test0 = utils.load_h5("ply_data_test0.h5")
test1 = utils.load_h5("ply_data_test1.h5")
# print(np.shape(test1[0]))

In [5]:
# aggregate test data, test label
test_data = np.append(test0[0], test1[0], axis=0)
test_labels = np.append(test0[1], test1[1], axis=0)
print(np.shape(test_data))
print(np.shape(test_labels))

(2468, 2048, 3)
(2468, 1)


In [6]:
train_labels_one_hot = []
for l in train_labels:
    one_hot = np.zeros(40, dtype=np.int)
    one_hot[l[0]] = 1
    train_labels_one_hot.append(one_hot)
train_labels_one_hot = np.array(train_labels_one_hot)
print(np.shape(train_labels_one_hot))

(9840, 40)


In [7]:
# one hot encode test_labels
test_labels_one_hot = []
for l in test_labels:
    one_hot = np.zeros(40, dtype=np.int)
    one_hot[l[0]] = 1
    test_labels_one_hot.append(one_hot)
test_labels_one_hot = np.array(test_labels_one_hot)
print(np.shape(test_labels_one_hot))

(2468, 40)


In [8]:
batch_size = 32

In [9]:
def input_transform(cloud):
    input_image = tf.expand_dims(cloud, -1)
    # B x 2048 x 3 x 1
    
    # 1st mlp layer
    i_trans_layer_conv1 = tf.contrib.layers.conv2d(inputs=input_image, num_outputs=64, kernel_size=[1, 3], padding="VALID", activation_fn=tf.nn.relu)
    # (B, 2048, 64)
    i_trans_layer_conv1 = tf.contrib.layers.batch_norm(inputs=i_trans_layer_conv1, decay=0.9)

    # 2nd mlp layer
    i_trans_layer_conv2 = tf.contrib.layers.conv2d(inputs=i_trans_layer_conv1, num_outputs=128, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
    # (B, 2048, 64)
    i_trans_layer_conv2 = tf.contrib.layers.batch_norm(inputs=i_trans_layer_conv2, decay=0.9)

    # 3rd mlp layer
    i_trans_layer_conv3 = tf.contrib.layers.conv2d(inputs=i_trans_layer_conv2, num_outputs=1024, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
    # (B, 2048, 64)
    i_trans_layer_conv3 = tf.contrib.layers.batch_norm(inputs=i_trans_layer_conv3, decay=0.9)

    # pooling
    i_max_pool = tf.contrib.layers.max_pool2d(inputs=i_trans_layer_conv3, kernel_size=[2048, 1], padding="VALID")

    i_max_pool = tf.reshape(i_max_pool, [batch_size, -1])
    
    # fnn1
    i_layer_fnn1 = tf.contrib.layers.fully_connected(inputs=i_max_pool, num_outputs=512, activation_fn=tf.nn.relu)
    print(np.shape(i_layer_fnn1))
    i_layer_fnn1 = tf.contrib.layers.batch_norm(inputs=i_layer_fnn1, decay=0.9)


    # fnn2
    i_layer_fnn2 = tf.contrib.layers.fully_connected(inputs=i_layer_fnn1, num_outputs=256, activation_fn=tf.nn.relu)
    i_layer_fnn2 = tf.contrib.layers.batch_norm(inputs=i_layer_fnn2, decay=0.9)


    K = 3
    weights = tf.get_variable('weights1', [256, 3*K], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    biases = tf.get_variable('bias1', [3*K], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    biases += tf.constant([1,0,0,0,1,0,0,0,1], dtype=tf.float32)
    transform = tf.matmul(i_layer_fnn2, weights)
    transform = tf.nn.bias_add(transform, biases)

    transform = tf.reshape(transform, [batch_size, 3, K])
#     transformed = tf.matmul(tf.squeeze(pt_cloud, [3]), transform)
    return transform

In [10]:
cloud_origin = tf.placeholder(tf.float32, [batch_size, 2048, 3])
print(cloud_origin)

Tensor("Placeholder:0", shape=(32, 2048, 3), dtype=float32)


In [11]:
# apply input transformation
t = input_transform(cloud_origin)
print(np.shape(t))
cloud = tf.matmul(cloud_origin, t)
print(np.shape(cloud))

(32, 512)
(32, 3, 3)
(32, 2048, 3)


In [12]:
def feature_transform(inputs):
    # B x 2048 x 3 x 1

    # 1st mlp layer
    f_trans_layer_conv1 = tf.contrib.layers.conv2d(inputs=inputs, num_outputs=64, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
    # (B, 2048, 1, 64)
    f_trans_layer_conv1 = tf.contrib.layers.batch_norm(inputs=f_trans_layer_conv1, decay=0.9)

    # 2nd mlp layer
    f_trans_layer_conv2 = tf.contrib.layers.conv2d(inputs=f_trans_layer_conv1, num_outputs=128, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
    # (B, 2048, 1, 64)
    f_trans_layer_conv2 = tf.contrib.layers.batch_norm(inputs=f_trans_layer_conv2, decay=0.9)

    # 3rd mlp layer
    f_trans_layer_conv3 = tf.contrib.layers.conv2d(inputs=f_trans_layer_conv2, num_outputs=1024, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
    # (B, 2048, 1, 64)
    f_trans_layer_conv3 = tf.contrib.layers.batch_norm(inputs=f_trans_layer_conv3, decay=0.9)

    # pooling
    f_max_pool = tf.contrib.layers.max_pool2d(inputs=f_trans_layer_conv3, kernel_size=[2048, 1], padding="VALID")

    f_max_pool = tf.reshape(f_max_pool, [batch_size, -1])
    
    # fnn1
    f_layer_fnn1 = tf.contrib.layers.fully_connected(inputs=f_max_pool, num_outputs=512, activation_fn=tf.nn.relu)
    f_layer_fnn1 = tf.contrib.layers.batch_norm(inputs=f_layer_fnn1, decay=0.9)


    # fnn2
    f_layer_fnn2 = tf.contrib.layers.fully_connected(inputs=f_layer_fnn1, num_outputs=256, activation_fn=tf.nn.relu)
    f_layer_fnn2 = tf.contrib.layers.batch_norm(inputs=f_layer_fnn2, decay=0.9)

    K = 64

    weights = tf.get_variable('weights2', [256, K*K], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    biases = tf.get_variable('bias2', [K*K], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    biases += tf.constant(np.eye(K).flatten(), dtype=tf.float32)
    transform = tf.matmul(f_layer_fnn2, weights)
    transform = tf.nn.bias_add(transform, biases)

    transform = tf.reshape(transform, [batch_size, K, K])
    return transform

In [13]:
# main network
pt_cloud = tf.expand_dims(cloud, -1)

# placeholder for one-hot labels
y = tf.placeholder(tf.float32, [None, 40])

# placeholder for labels
y_labels = tf.placeholder(tf.int64, [None])

# 1st mlp layer
layer_conv1 = tf.contrib.layers.conv2d(inputs=pt_cloud, num_outputs=64, kernel_size=[1, 3], padding="VALID", activation_fn=tf.nn.relu)
layer_conv1 = tf.contrib.layers.batch_norm(layer_conv1)

# 2nd mlp layer
layer_conv2 = tf.contrib.layers.conv2d(inputs=layer_conv1, num_outputs=64, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
layer_conv2_norm = tf.contrib.layers.batch_norm(layer_conv2)

# apply feature transformation
f_transform = feature_transform(layer_conv2_norm)
layer_conv2 = tf.matmul(tf.squeeze(layer_conv2_norm, [2]), f_transform)
layer_conv2 = tf.expand_dims(layer_conv2, [2])

# 3rd mlp layer
layer_conv3 = tf.contrib.layers.conv2d(inputs=layer_conv2, num_outputs=64, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
layer_conv3 = tf.contrib.layers.batch_norm(layer_conv3)


# 4th cnn
layer_conv4 = tf.contrib.layers.conv2d(inputs=layer_conv3, num_outputs=128, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
layer_conv4 = tf.contrib.layers.batch_norm(layer_conv4)


# 5th cnn
layer_conv5 = tf.contrib.layers.conv2d(inputs=layer_conv4, num_outputs=1024, kernel_size=[1, 1], padding="VALID", activation_fn=tf.nn.relu)
layer_conv5 = tf.contrib.layers.batch_norm(layer_conv5)

# max pooling
max_pool = tf.contrib.layers.max_pool2d(inputs=layer_conv5, kernel_size=[2048, 1], padding="VALID")


# fnn1
layer_fnn1 = tf.contrib.layers.fully_connected(inputs=max_pool, num_outputs=512, activation_fn=tf.nn.relu)
layer_fnn1 = tf.contrib.layers.batch_norm(layer_fnn1)

# fnn2
layer_fnn2 = tf.contrib.layers.fully_connected(inputs=layer_fnn1, num_outputs=256, activation_fn=tf.nn.relu)
layer_fnn2 = tf.contrib.layers.batch_norm(layer_fnn2)

layer_fnn2 = tf.contrib.layers.dropout(inputs=layer_fnn2, keep_prob=0.7)

# fnn3
logits = tf.contrib.layers.fully_connected(inputs=layer_fnn2, num_outputs=40, activation_fn=tf.nn.relu)
logits = tf.squeeze(logits, [1, 2])

# softmax
output = tf.nn.softmax(logits)
output_class = tf.argmax(output,axis=1)

In [15]:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.001
l_rate = tf.train.exponential_decay(starter_learning_rate, global_step, int(20*307), 0.5, staircase=True)
reg_weight = 0.001

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

# 
# f_transform = end_points['transform'] # BxKxK
# K = 64
# mat_diff = tf.matmul(f_transform, tf.transpose(f_transform, perm=[0,2,1]))
# mat_diff -= tf.constant(np.eye(K), dtype=tf.float32)
# mat_diff_loss = tf.nn.l2_loss(mat_diff) 

# loss = loss + mat_diff_loss * reg_weight

# optimizer
optim = tf.train.AdamOptimizer(learning_rate=l_rate)
optimizer = optim.minimize(loss, global_step=global_step)

In [16]:
# accuracy
correct_labels = tf.equal(output_class, y_labels)
accuracy = tf.reduce_mean(tf.cast(correct_labels, tf.float32))

In [17]:
from random import *
import math
# only rotate against y axis
def rotate(pt_cloud):
    angle = np.radians(randint(0, 360))
    R = np.array([[math.cos(angle), 0, math.sin(angle)], [0, 1, 0], [-math.sin(angle), 0, math.cos(angle)]])
    rotated_pt_cloud = []
    for p in pt_cloud:
        rotated_pt_cloud.append((np.matmul(R, p)))
    return np.array(rotated_pt_cloud)

In [18]:
def jitter(pt_cloud):
    pt_cloud = pt_cloud + np.random.normal(0, 0.02, (2048, 3))
    return pt_cloud

In [19]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# print('Learning rate: %f' % (sess.run(optim ._lr)))

num_iter = int(100*307)
# num_iter = int(1000*batch_size)

for i in range(num_iter):
    idx = np.random.choice(9840, [batch_size], False)
    batch_img_vanilla = train_data[idx][:]
    batch_y = train_labels_one_hot[idx][:]
    # data augmentation
#     batch_img_rotate = [rotate(img) for img in batch_img_vanilla]
#     batch_img = np.array([jitter(img) for img in batch_img_rotate])
#     batch_y_labels = np.array([label.argmax() for label in batch_y])
    _, l= sess.run([optimizer, loss], feed_dict = {cloud_origin: batch_img_vanilla , y: batch_y})
    if i % 307 == 0:
        print(l)
        # save_path = tf.train.Saver().save(sess, os.path.join("model/", "model.ckpt"))
        # print("Model saved in file: %s" % save_path)

4.3623
1.21246
0.808551
0.923609
0.314016
0.576825
0.309686
0.264848
0.841429
0.65341
0.720496
0.675332
0.196361
0.511184
0.463583
0.406917
0.278022
0.256289
0.203865
0.149685
0.248356
0.200991
0.398955
0.292992
0.371526
0.197388
0.137973
0.166174
0.108089
0.115245
0.0622786
0.437801
0.381864
0.179708
0.178001
0.120153
0.147666
0.235699
0.085003
0.0556461
0.0616621
0.0144146
0.0938984
0.0927649
0.0310371
0.0252497
0.0733644
0.175047
0.0519938
0.0873374
0.17606
0.0864098
0.0167059
0.0424972
0.0365879
0.379465
0.0783945
0.0221556
0.0508381
0.0182925
0.00458056
0.0144775
0.108823
0.0841691
0.0685509
0.0646207
0.031422
0.0531342
0.0520339
0.0022836
0.0122519
0.00869116
0.0128309
0.123395
0.131998
0.00512941
0.0172462
0.0479059
0.0425948
0.0506812
0.0491047
0.00153749
0.0137139
0.000531841
0.0370705
0.0736768
0.0322824
0.0321427
0.084366
0.0136217
0.0122896
0.00808755
0.00882658
0.008449
0.0679349
0.0243446
0.0031865


In [20]:
right_count = 0
i = 0
while i < 2464:
    j = min(i + 32, len(test_data))
    correct_labels = tf.equal(output_class, y_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_labels, tf.float32))
    # compute accuracy on test data
    labels = np.array([label.argmax() for label in test_labels_one_hot[i:j]])
    #print(np.shape(labels))
    accuracy = sess.run([accuracy],feed_dict = {cloud: test_data[i:j], y: test_labels_one_hot[i:j], y_labels: labels})
    print ("Accuracy from " ,i, j, accuracy)
    right_count = right_count + accuracy[0] * (j - i)
    i += 32
final_accuracy = right_count / 2464
print("Final Accuracy", final_accuracy)

('Accuracy from ', 0, 32, [0.71875])
('Accuracy from ', 32, 64, [0.5])
('Accuracy from ', 64, 96, [0.5625])
('Accuracy from ', 96, 128, [0.75])
('Accuracy from ', 128, 160, [0.78125])
('Accuracy from ', 160, 192, [0.59375])
('Accuracy from ', 192, 224, [0.59375])
('Accuracy from ', 224, 256, [0.71875])
('Accuracy from ', 256, 288, [0.5625])
('Accuracy from ', 288, 320, [0.6875])
('Accuracy from ', 320, 352, [0.6875])
('Accuracy from ', 352, 384, [0.65625])
('Accuracy from ', 384, 416, [0.5625])
('Accuracy from ', 416, 448, [0.75])
('Accuracy from ', 448, 480, [0.8125])
('Accuracy from ', 480, 512, [0.75])
('Accuracy from ', 512, 544, [0.65625])
('Accuracy from ', 544, 576, [0.75])
('Accuracy from ', 576, 608, [0.78125])
('Accuracy from ', 608, 640, [0.59375])
('Accuracy from ', 640, 672, [0.59375])
('Accuracy from ', 672, 704, [0.5625])
('Accuracy from ', 704, 736, [0.65625])
('Accuracy from ', 736, 768, [0.5625])
('Accuracy from ', 768, 800, [0.6875])
('Accuracy from ', 800, 832, [0.6

In [ ]:
acc = []
while((i+1) * batch_size < len(X_test)):
    acc.append(sess.run(accuracy, feed_dict={X: X_test[i*batch_size:(i+1)*batch_size], y_: y_test[i*batch_size:(i+1)*batch_size]}))
    i+=1
acc.append(sess.run(accuracy, feed_dict={X: X_test[i*batch_size:], y_: y_test[i*batch_size:]}))

In [23]:
right_count = 0
i = 0
while i < len(train_data):
    j = min(i + 32, len(train_data))
    correct_labels = tf.equal(output_class, y_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_labels, tf.float32))
    # compute accuracy on test data
    labels = np.array([label.argmax() for label in train_labels_one_hot[i:j]])
    #print(np.shape(labels))
    accuracy = sess.run([accuracy],feed_dict = {cloud: train_data[i:j], y: train_labels_one_hot[i:j], y_labels: labels})
    print ("Accuracy from " ,i, j, accuracy)
    right_count = right_count + accuracy[0] * (j - i)
    i += 32
final_accuracy = right_count / len(train_data)
print("Final Accuracy", final_accuracy)

Accuracy from  0 32 [0.3125]
Accuracy from  32 64 [0.28125]
Accuracy from  64 96 [0.28125]
Accuracy from  96 128 [0.34375]
Accuracy from  128 160 [0.25]
Accuracy from  160 192 [0.1875]
Accuracy from  192 224 [0.34375]
Accuracy from  224 256 [0.1875]
Accuracy from  256 288 [0.21875]
Accuracy from  288 320 [0.21875]
Accuracy from  320 352 [0.15625]
Accuracy from  352 384 [0.3125]
Accuracy from  384 416 [0.3125]
Accuracy from  416 448 [0.3125]
Accuracy from  448 480 [0.25]
Accuracy from  480 512 [0.25]
Accuracy from  512 544 [0.1875]
Accuracy from  544 576 [0.21875]
Accuracy from  576 608 [0.1875]
Accuracy from  608 640 [0.34375]
Accuracy from  640 672 [0.1875]
Accuracy from  672 704 [0.375]
Accuracy from  704 736 [0.1875]
Accuracy from  736 768 [0.375]
Accuracy from  768 800 [0.28125]
Accuracy from  800 832 [0.1875]
Accuracy from  832 864 [0.3125]
Accuracy from  864 896 [0.25]
Accuracy from  896 928 [0.3125]
Accuracy from  928 960 [0.28125]
Accuracy from  960 992 [0.1875]
Accuracy from  

Accuracy from  7808 7840 [0.375]
Accuracy from  7840 7872 [0.15625]
Accuracy from  7872 7904 [0.21875]
Accuracy from  7904 7936 [0.21875]
Accuracy from  7936 7968 [0.34375]
Accuracy from  7968 8000 [0.40625]
Accuracy from  8000 8032 [0.25]
Accuracy from  8032 8064 [0.125]
Accuracy from  8064 8096 [0.25]
Accuracy from  8096 8128 [0.21875]
Accuracy from  8128 8160 [0.25]
Accuracy from  8160 8192 [0.21875]
Accuracy from  8192 8224 [0.375]
Accuracy from  8224 8256 [0.40625]
Accuracy from  8256 8288 [0.25]
Accuracy from  8288 8320 [0.34375]
Accuracy from  8320 8352 [0.375]
Accuracy from  8352 8384 [0.3125]
Accuracy from  8384 8416 [0.3125]
Accuracy from  8416 8448 [0.25]
Accuracy from  8448 8480 [0.3125]
Accuracy from  8480 8512 [0.3125]
Accuracy from  8512 8544 [0.15625]
Accuracy from  8544 8576 [0.25]
Accuracy from  8576 8608 [0.28125]
Accuracy from  8608 8640 [0.28125]
Accuracy from  8640 8672 [0.34375]
Accuracy from  8672 8704 [0.21875]
Accuracy from  8704 8736 [0.21875]
Accuracy from  

ValueError: Cannot feed value of shape (16, 2048, 3) for Tensor 'MatMul_1:0', which has shape '(32, 2048, 3)'